In [4]:
import sys
sys.path.append("../")

%load_ext autoreload
%autoreload 2

In [7]:
import pickle
import re
import tweepy
import json
import time
from datetime import datetime
from tqdm.auto import tqdm, trange
import requests
import glob
import urllib.parse
import nltk
from labelling_classifier import update_model
from utills import edit_distance, extract_twitter_username
from data_collection_methods import query_reddit_comments, query_reddit_submissions

In [10]:
DATA_PATH = '../data/twitter_reddit/'

In [ ]:

def print_highlighted_text(text):
    print(re.sub(r"(my twitter account|twitter.com)", '\x1b[31m\g<1>\x1b[0m', text,  flags=re.IGNORECASE))

In [29]:
after = str(int(datetime(2010, 1, 1).timestamp()))
before = str(int(datetime.now().timestamp()))
query = '"my twitter account"+"twitter.com"'

In [11]:
with open(DATA_PATH + 'to_collect.p', 'rb') as f:
    (to_collect, skipped, already_scanned, before, after) = pickle.load(f)

In [12]:
len(to_collect), len(skipped), len(already_scanned), before, after

(2218, 33, 2231, '1622733540', 1622579527)

In [95]:
# # to_collect = []
# skipped = []
# already_scanned = set()

Search in Reddit Comments
===

In [16]:
new_negatives = []
new_positives = []

In [ ]:
while(True):
    clf = update_model(new_positives, new_negatives)
    new_negatives = []
    new_positives = []
    
    res = query_reddit_comments(query, before, after)
    for r in res:
        if r['author'] in already_scanned:
            continue
        print('-' * 20)
        twitter_un = extract_username(r['body'])
        dist = edit_distance(twitter_un, r['author'])
        if twitter_un == 'i':
            print('Bad twitter username. Skipping...')
            continue
        p = clf.predict_proba([r])[0, 1]
        print('Prediction:', p)
        if dist < 0.01:
            print('Reddit username:', r['author'])
            
            print('Twitter username:', twitter_un)
            print('Auto added!')
            to_collect.append((r['author'], twitter_un, r))
            already_scanned.add(r['author'])
            new_positives.append(r)
            continue
        if 'not my twitter' in r['body'].lower():
            print('Auto skipped!')
            new_negatives.append(r)
            continue
        print_highlighted_text(r['body'])
        print('Reddit username:', r['author'])
        twitter_un = extract_username(r['body'])
        print('Twitter username:', twitter_un)
        print('Edit Distance:', dist)
        response = input('Save:1, Skip:0, Save-for-later:2, Blacklist Name:3')
        if response == '1':
            to_collect.append((r['author'], twitter_un, r))
            already_scanned.add(r['author'])
            new_positives.append(r)
        elif response == '2':
            skipped.append(r)
        elif response == '0':
            new_negatives.append(r)
            continue
        elif response == '3':
            already_scanned.add(r['author'])
    after = r['created_utc']            
    with open(DATA_PATH + 'to_collect.p', 'wb') as f:
        pickle.dump((to_collect, skipped, already_scanned, before, after), f)

Do the search with Reddit submissions
===

In [37]:
after = str(int(datetime(2010, 1, 1).timestamp()))
before = str(int(datetime.now().timestamp()))
query = '"my twitter account"+"twitter.com"'

In [ ]:
while(True):
    # For now lets just see what the label classifying model syas, it is not accurate enough to use it
    clf = update_model(new_positives, new_negatives)
    new_negatives = []
    new_positives = []
    
    res = query_reddit_submissions(query, before, after)
    for r in res:
        # Workaround so that comments and submissions are interchangable in the rest of the code
        r['body'] = r['selftext']
        if r['author'] in already_scanned:
            continue
        print('-' * 20)
        twitter_un = extract_username(r['body'])
        dist = edit_distance(twitter_un, r['author'])
        if twitter_un == 'i' or twitter_un is None:
            print('Bad twitter username. Skipping...')
            continue
        p = clf.predict_proba([r])[0, 1]
        
        
        if dist < 0.01:
            print('Reddit username:', r['author'])
            
            print('Twitter username:', twitter_un)
            print('Auto added!')
            to_collect.append((r['author'], twitter_un, r))
            already_scanned.add(r['author'])
            new_positives.append(r)
            continue
        if 'not my twitter' in r['body'].lower():
            print('Auto skipped!')
            new_negatives.append(r)
            continue
        print('Title: ', r['title'])
        print_highlighted_text(r['body'])
        print('Reddit username:', r['author'])
        twitter_un = extract_username(r['body'])
        print('Twitter username:', twitter_un)
        print('Edit Distance:', dist)
        print('Prediction:', p)
        response = input('Save:1, Skip:0, Save-for-later:2, Blacklist Name:3')
        if response == '1':
            to_collect.append((r['author'], twitter_un, r))
            already_scanned.add(r['author'])
            new_positives.append(r)
        elif response == '2':
            skipped.append(r)
        elif response == '0':
            new_negatives.append(r)
            continue
        elif response == '3':
            already_scanned.add(r['author'])
    after = r['created_utc']            
    with open(DATA_PATH + 'to_collect.p', 'wb') as f:
        pickle.dump((to_collect, skipped, already_scanned, before, after), f)